In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [1]:
import urllib
import pandas as pd
import os

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\dalwoo\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [3]:
from selenium import webdriver    # 라이브러리에서 사용하는 모듈만 호출
from urllib.request import urlopen
from bs4 import BeautifulSoup as BS
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
from urllib.parse import quote_plus    # 한글 텍스트를 퍼센트 인코딩으로 변환
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   # 해당 태그를 기다림
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException    # 태그가 없는 예외 처리

import matplotlib.pyplot as plt
%matplotlib inline

### 문제1 - 스타벅스 DF

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from selenium import webdriver

url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
page = urlopen(url)
soup = BS(page,'html.parser')

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())
driver.get(url)



====== WebDriver manager ======
Current chromium version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\dalwoo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [5]:
time.sleep(3) # 페이지 로딩 시간
seoul_path = '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'
in_seoul = driver.find_element_by_xpath(seoul_path)
in_seoul.click() # seoul 지역만 볼 수 있도록 클릭

time.sleep(3) # 페이지 로딩 시간
total_path = '//*[@id="mCSB_2_container"]/ul/li[1]/a'
in_total = driver.find_element_by_xpath(total_path) 
driver.execute_script("arguments[0].click();", in_total) # 서울내 지역 전체 검색

In [6]:
time.sleep(4)
req = driver.page_source # 현재 페이지 소스(서울지역) 다시 저장
soup = BS(req,"html.parser")

In [7]:
seoul_list = soup.find_all("ul","quickSearchResultBoxSidoGugun")
seoul_list

[<ul class="quickSearchResultBoxSidoGugun"><li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li><li class="quickResultLstCon" data-code="3672" data-hlytag="null" data-index="1" data-lat="37.510178" data-long="127.022223" data-name="논현역사거리" data-storecd="1434" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="논현역사거리" data-store="1434" data-yn="N">논현역사거리  </strong> <p class="result_details">서울특별시 강남구 강남대로 538 (논현동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li><li class="quickResultLstCon" data-code="3858" data-hlytag="null" data-index="2" data-lat="37.514132" data-long="127.020563" data-name="신사역성일

In [8]:
def search_gu(obj): # text에서 구 정보만 뺴내는 함수
    result = ""
    gu = ["강동구","강남구","강북구","강서구",'관악구',
          "광진구","구로구","금천구","노원구","도봉구",
          "동대문구","동작구","마포구","서대문구","서초구",
          "성동구","성북구","송파구","양천구","영등포구","용산구",
          "은평구","종로구","중구","중랑구"]
    count = 0
    for i in gu:
        if i in obj:
            result = i
            count += 1
        if count > 1:
            print("구 2개? 이상발생")
    return result

In [9]:
soup.select(".quickResultLstCon > strong")[0].get_text()

'역삼아레나빌딩  '

In [10]:
soup.select(".quickResultLstCon > p")[0].get_text()

'서울특별시 강남구 언주로 425 (역삼동)1522-3232'

In [11]:
result_df = pd.DataFrame()
result_df = result_df.reindex(columns = ['매장이름','주소','구'])

In [12]:
flag = True
i = 0
while flag:
    try:
        m = soup.select(".quickResultLstCon > strong")[i].get_text()
        j = soup.select(".quickResultLstCon > p")[i].get_text().rstrip(' ') # 우측 공백 제거
        g = search_gu(j)
        result_df = result_df.append({'매장이름':m,'주소':j,'구':g},ignore_index=True)
        print(m,j,g)
        i += 1
    except:
        excep +=1 # 예외 발생 수
        print('매장 수 : ', i+1)
        flag = False

역삼아레나빌딩   서울특별시 강남구 언주로 425 (역삼동)1522-3232 강남구
논현역사거리   서울특별시 강남구 강남대로 538 (논현동)1522-3232 강남구
신사역성일빌딩   서울특별시 강남구 강남대로 584 (논현동)1522-3232 강남구
국기원사거리   서울특별시 강남구 테헤란로 125 (역삼동)1522-3232 강남구
스탈릿대치R   서울특별시 강남구 남부순환로 2947 (대치동)1522-3232 강남구
봉은사역   서울특별시 강남구 봉은사로 619 (삼성동)1522-3232 강남구
압구정윤성빌딩   서울특별시 강남구 논현로 834 (신사동)1522-3232 강남구
코엑스별마당   서울특별시 강남구 영동대로 513 (삼성동)1522-3232 강남구
삼성역섬유센터R   서울특별시 강남구 테헤란로 518 (대치동)1522-3232 강남구
압구정R   서울특별시 강남구 언주로 861 (신사동)1522-3232 강남구
수서역R   서울특별시 강남구 광평로 281 (수서동)1522-3232 강남구
양재강남빌딩R   서울특별시 강남구 남부순환로 2621 (도곡동)1522-3232 강남구
선릉동신빌딩R   서울특별시 강남구 테헤란로 409 (삼성동)1522-3232 강남구
봉은사로선정릉   서울특별시 강남구 봉은사로 446 (삼성동)1522-3232 강남구
강남오거리   서울특별시 강남구 봉은사로2길 39 (역삼동)1522-3232 강남구
스타필드코엑스몰R   서울특별시 강남구 영동대로 513 (삼성동) 코엑스 A106호1522-3232 강남구
강남구청정문   서울특별시 강남구 학동로 419 (청담동)1522-3232 강남구
도곡공원   서울특별시 강남구 도곡로 205 (역삼동)1522-3232 강남구
강남R   서울특별시 강남구 강남대로 390 (역삼동)1522-3232 강남구
대치은마사거리   서울특별시 강남구 도곡로 457 (대치동)1522-3232 강남구
청담영동대로   서울특별시 강남구 영동대로 720 (청담동)1522-3232 강남구
압구정  

NameError: name 'excep' is not defined

In [13]:
result_df

,매장이름,주소,구
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동)1522-3232,강남구
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동)1522-3232,강남구
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동)1522-3232,강남구
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동)1522-3232,강남구
4,스탈릿대치R,서울특별시 강남구 남부순환로 2947 (대치동)1522-3232,강남구
...,...,...,...
561,사가정역,서울특별시 중랑구 면목로 3101522-3232,중랑구
562,상봉역,서울특별시 중랑구 망우로 307 (상봉동)1522-3232,중랑구
563,묵동이마트,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)152...",중랑구
564,묵동,서울특별시 중랑구 동일로 952 1522-3232,중랑구


In [14]:
result_df.주소 = result_df.주소.apply(lambda x: x[:-9]) # 마지막 9글자 제거(전화번호)
result_df

,매장이름,주소,구
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),강남구
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),강남구
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),강남구
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),강남구
4,스탈릿대치R,서울특별시 강남구 남부순환로 2947 (대치동),강남구
...,...,...,...
561,사가정역,서울특별시 중랑구 면목로 310,중랑구
562,상봉역,서울특별시 중랑구 망우로 307 (상봉동),중랑구
563,묵동이마트,"서울특별시 중랑구 동일로 932 (묵동, 묵동자이아파트) (묵동이마트 B1층)",중랑구
564,묵동,서울특별시 중랑구 동일로 952,중랑구


### 문제2 - 이디야 DF

In [15]:
url = 'https://www.ediya.com/contents/find_store.html#c'
page = urlopen(url)
soup = BS(page,'html.parser')

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())
driver.get(url)



====== WebDriver manager ======
Current chromium version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\dalwoo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [16]:
gu = ["강동구","강남구","강북구",'관악구',
          "광진구","구로구","금천구","노원구","도봉구",
          "동대문구","동작구","마포구","서대문구","서초구",
          "성동구","성북구","송파구","양천구","영등포구","용산구",
          "은평구","종로구","중랑구"]
error_gu = ['서울 강서구',"서울 중구"] # 검색결과가 너무 많음 -> 서울로 이름을 바꾸니 성공

In [17]:
juso_search_path = '//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a' # 주소검색 버튼
driver.find_element_by_xpath(juso_search_path).click() # 버튼 클릭

click_input_path = '//*[@id="keyword"]'
elem = driver.find_element_by_xpath(click_input_path) # 검색창 html

ediya_df = pd.DataFrame()
ediya_df = ediya_df.reindex(columns = ['매장이름','주소','구'])
count=0 # 구 검색완료한 갯수
error_gu = []    

try:
    for i in gu:
        i_cn = 0
        elem.send_keys(i) # 입력 키워드
        elem.send_keys(Keys.RETURN) # 엔터
        time.sleep(7) # 페이지가 다 로드될때까지 대기
        req = driver.page_source # 현재 검색구 페이지 소스 저장
        soup = BS(req,"html.parser")        
        gu_count = len(soup.select(".result_list > ul > li > a > dl > dt")) # 각 구별 매장 개수
        for k in range(gu_count):
            m = soup.select(".result_list > ul > li > a > dl > dt")[k].get_text() # 이름
            j = soup.select(".result_list > ul > li > a > dl > dd")[k].get_text() # 주소
            ediya_df = ediya_df.append({'매장이름':m,'주소':j,'구':i},ignore_index=True)
            i_cn +=1 # 잘 되는지 확인하기위해 매장갯수변수를 추가 생성
        elem.clear() # 다 찾고나면 입력창을 비운다
        print(i,"의 매장수 : ", i_cn)        
except: # 검색결과가 너무 많습니다 error 발생
    error_gu = error_gu.append(i) # 이 방식으로 강서구, 중구 발견
    pass

강동구 의 매장수 :  25
강남구 의 매장수 :  45
강북구 의 매장수 :  20
관악구 의 매장수 :  29
광진구 의 매장수 :  24
구로구 의 매장수 :  29
금천구 의 매장수 :  24
노원구 의 매장수 :  31
도봉구 의 매장수 :  23
동대문구 의 매장수 :  22
동작구 의 매장수 :  23
마포구 의 매장수 :  39
서대문구 의 매장수 :  21
서초구 의 매장수 :  30
성동구 의 매장수 :  23
성북구 의 매장수 :  32
송파구 의 매장수 :  40
양천구 의 매장수 :  27
영등포구 의 매장수 :  41
용산구 의 매장수 :  15
은평구 의 매장수 :  26
종로구 의 매장수 :  31
중랑구 의 매장수 :  26


In [18]:
for i in error_gu:
    i_cn = 0
    elem.send_keys(i) # 입력 키워드
    elem.send_keys(Keys.RETURN) # 엔터
    time.sleep(7) # 페이지가 다 로드될때까지 대기
    req = driver.page_source # 현재 검색구 페이지 소스 저장
    soup = BS(req,"html.parser")        
    gu_count = len(soup.select(".result_list > ul > li > a > dl > dt")) # 각 구별 매장 개수
    for k in range(gu_count):
        m = soup.select(".result_list > ul > li > a > dl > dt")[k].get_text() # 이름
        j = soup.select(".result_list > ul > li > a > dl > dd")[k].get_text() # 주소
        ediya_df = ediya_df.append({'매장이름':m,'주소':j,'구':i},ignore_index=True)
        i_cn +=1 # 잘 되는지 확인하기위해 매장갯수변수를 추가 생성
    elem.clear() # 다 찾고나면 입력창을 비운다
    print(i,"의 매장수 : ", i_cn)       

In [19]:
ediya_df

,매장이름,주소,구
0,강동SK허브점,"서울 강동구 천호대로 1107 (길동, 강동역에스케이허브)",강동구
1,강동구청점,"서울 강동구 성내로6길 20 (성내동, 청일 베르네)",강동구
2,강동역점,"서울 강동구 천호대로 1065 (천호동, 강동상떼빌)",강동구
3,강일점,"서울 강동구 아리수로93길 19 (강일동, 임페리얼타워)",강동구
4,고덕그라시움후문점,서울 강동구 동남로82길 121 (고덕동),강동구
...,...,...,...
641,중랑교차로점,서울 중랑구 동일로 683 (면목동),중랑구
642,중랑역점,서울 중랑구 망우로 198 (상봉동),중랑구
643,중화동점,서울 중랑구 동일로129길 1 (중화동),중랑구
644,중화역점,"서울 중랑구 동일로 815, 1층",중랑구


### 문제3 - 위치 분포 비교분석

> 스타벅스 위도 및 경도

In [24]:
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
page = urlopen(url)
soup = BS(page,'html.parser')
driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())
driver.get(url)



====== WebDriver manager ======
Current chromium version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\dalwoo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [28]:
time.sleep(3) # 페이지 로딩 시간
seoul_path = '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'
in_seoul = driver.find_element_by_xpath(seoul_path)
in_seoul.click() # seoul 지역만 볼 수 있도록 클릭

time.sleep(3) # 페이지 로딩 시간
total_path = '//*[@id="mCSB_2_container"]/ul/li[1]/a'
in_total = driver.find_element_by_xpath(total_path) 
driver.execute_script("arguments[0].click();", in_total) # 서울내 지역 전체 검색

req = driver.page_source # 현재 페이지 소스(서울지역) 다시 저장
soup = BS(req,"html.parser")
name = soup.select('li[data-name]')
starbucks = []

time.sleep(1)
for name_one in name:
    x = name_one['data-lat'] # 위도 저장
    y = name_one['data-long'] # 경도 저장
    starbucks.append({'lat': x, 'long':y}) # dict 형태로 리스트에 저장

In [29]:
starbucks = pd.DataFrame(starbucks)
starbucks

,lat,long
0,37.501087,127.043069
1,37.510178,127.022223
2,37.514132,127.020563
3,37.499517,127.031495
4,37.494668,127.062583
...,...,...
561,37.579594,127.087966
562,37.59689,127.08647
563,37.613433,127.077484
564,37.615368,127.076633


> 이디야 위도 및 경도

In [30]:
url = 'https://www.ediya.com/contents/find_store.html#c'
page = urlopen(url)
soup = BS(page,'html.parser')

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())
driver.get(url)



====== WebDriver manager ======
Current chromium version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\dalwoo\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


In [31]:
ediya_gu = ["강동구","강남구","강북구",'관악구',
          "광진구","구로구","금천구","노원구","도봉구",
          "동대문구","동작구","마포구","서대문구","서초구",
          "성동구","성북구","송파구","양천구","영등포구","용산구",
          "은평구","종로구","중랑구",'서울 강서구',"서울 중구"] 

In [32]:
juso_search_path = '//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a' # 주소검색 버튼
driver.find_element_by_xpath(juso_search_path).click() # 버튼 클릭
click_input_path = '//*[@id="keyword"]'
elem = driver.find_element_by_xpath(click_input_path) # 검색창 html

ediya = []


for i in ediya_gu:
    elem.send_keys(i) # 입력 키워드
    elem.send_keys(Keys.RETURN) # 엔터
    time.sleep(7) # 페이지가 다 로드될때까지 대기
    req = driver.page_source # 현재 검색구 페이지 소스 저장
    soup = BS(req,"html.parser")        
    gu_count = len(soup.select(".result_list > ul > li > a > dl > dt")) # 각 구별 매장 개수
    name = soup.select(".result_list > ul > li > a[onclick]")
    for k in name:
        try:
            info = k.get("onclick").rstrip(';fnMove();').split("('")[1].split(",")
            x = float(info[1].strip("'")) # 위도 저장
            y = float(info[0].strip("'")) # 경도 저장
            ediya.append({'lat': x, 'long':y}) # dict 형태로 리스트에 저장
        except:
            print(info[0].strip("'")) # html 문법이 다른 경우
    elem.clear() # 다 찾고나면 입력창을 비운다 

서울 강동구 천호대로 1132 (성내동)
서울 강동구 양재대로 1471 (길동)
서울 강동구 명일로 200 (길동)
서울 강동구 양재대로 1377 (성내동)
서울 강동구 명일로 110 (둔촌동
서울 강동구 천호대로 1027 (천호동)
서울 강동구 천호대로 1006 (성내동)
서울 강동구 올림픽로 665 (천호동)
서울 강남구 도산대로37길 20 (신사동)
서울 강남구 언주로 711 (논현동)
서울 강남구 논현동 
서울 강남구 테헤란로88길 10 (대치동)
서울 강남구 봉은사로 442 (삼성동)
서울 강남구 테헤란로107길 11 (삼성동)
서울 강남구 선릉로161길 20 (신사동)
서울 강남구 신사동
서울 강남구 논현로63길 16 (역삼동
서울 강남구 논현로98길 7 (역삼동)
서울 강남구 테헤란로4길 46 (역삼동
서울 강남구 개포로 218 (개포동)
서울 강북구 삼양로 501 (수유동)
서울 강북구 한천로 1083 (수유동)
서울 강북구 삼양로 118 (미아동)
서울 강북구 삼양로 293 (수유동)
서울 강북구 덕릉로 82 (수유동
서울 강북구 도봉로 355 (수유동)
서울 관악구 관악로 164 (봉천동
서울 관악구 남부순환로 1915 (봉천동)
서울 관악구 과천대로 909 (남현동
서울 관악구 신림로 91 (신림동)
서울 관악구 보라매로 62 (봉천동
서울 관악구 은천로 23 (봉천동)
서울 관악구 남부순환로 1725 (봉천동)
서울 관악구 남부순환로 1874 (봉천동)
서울 관악구 과천대로 951 (남현동)
서울 관악구 호암로26길 17 (신림동)
서울 관악구 신림로 262-1 (신림동)
서울 관악구 신원로 16 (신림동)
서울 광진구 아차산로 229 (화양동
서울 광진구 자양로 136 (구의동)
서울 광진구 아차산로 481 (구의동)
서울 광진구 광나루로 508 (구의동)
서울 광진구 능동로 290 (능동
서울 광진구 천호대로 663 (구의동)
서울 광진구 자양로 46 (자양동)
서울 광진구 자양번영로 34 (자양동
서울 광진구 용마산로 111 (중곡동

In [33]:
soup.select(".result_list > ul > li > a[onclick]")[0].get("onclick")

"panAddTo('서울 중구 세종대로1길 11 (봉래동1가)','0');fnMove();"

In [34]:
soup.select(".result_list > ul > li > a[onclick]")[0].get("onclick").rstrip(';fnMove();').split("('")[1].split(",")

["서울 중구 세종대로1길 11 (봉래동1가)'", "'0'"]

### 전처리 Task

> 위도 경도 정보가 사이트에 순수하게 없어서 외부 데이터가 필요한 경우
>> 1. 위도 경도 정보가 들어는 있으나 0,0으로 들어있음
>> 2. 아예 들어있지 않음

In [35]:
ediya

[{'lat': 0.0, 'long': 0.0},
 {'lat': 0.0, 'long': 0.0},
 {'lat': 0.0, 'long': 0.0},
 {'lat': 37.565355270810386, 'long': 127.17410624484566},
 {'lat': 37.56039666265223, 'long': 127.16370018873648},
 {'lat': 37.55845827938167, 'long': 127.15111510357119},
 {'lat': 37.55413723208973, 'long': 127.15518054387726},
 {'lat': 0.0, 'long': 0.0},
 {'lat': 37.5506474264789, 'long': 127.144288436536},
 {'lat': 37.5462992589271, 'long': 127.145789262651},
 {'lat': 37.55139001231505, 'long': 127.17398322022565},
 {'lat': 37.55445806224201, 'long': 127.14074049194033},
 {'lat': 37.5508657313861, 'long': 127.128126213246},
 {'lat': 37.5513635720856, 'long': 127.134728442286},
 {'lat': 0.0, 'long': 0.0},
 {'lat': 37.54016837687974, 'long': 127.12456936024019},
 {'lat': 37.5467270340876, 'long': 127.134809493652},
 {'lat': 0.0, 'long': 0.0},
 {'lat': 37.51654171724045, 'long': 127.0401601992311},
 {'lat': 37.51408005446769, 'long': 127.02810578707653},
 {'lat': 37.50133876179308, 'long': 127.052429282

In [ ]:
### 2021-10-07 이후 나머지 작업 예정